# Introduction to Climatrix and Basic Operations

This notebook demonstrates basic operations using the climatrix library, including creating a BaseClimatrixDataset object, accessing domain and axis information, and performing basic arithmetic operations.

In [51]:
import climatrix as cm
import xarray as xr
import numpy as np
from datetime import datetime

## Creation of simple Dataset

In [52]:
xarray_da = xr.DataArray(
    data=np.random.rand(1, 5, 5),
    dims=("time", "latitude", "longitude"),
    coords={
        "time": np.array(
            [
                datetime(2000, 1, 1),
            ],
            dtype="datetime64",
        ),
        "latitude": (
            ("latitude",),
            np.array([-90, -45, 0, 45, 90]),
        ),
        "longitude": (
            ("longitude",),
            np.array([-180, -90, 0, 90, 180]),
        ),
    },
)
cm_dense = xarray_da.to_dataset(name="temperature").cm
cm_dense

### Information about the `Domain`

In [53]:
domain = cm_dense.domain

print(f"is the domain sparse? {domain.is_sparse}")
print(f"is the domain dynamic? {domain.is_dynamic}")

is the domain sparse? False
is the domain dynamic? False


### Accessing Axes
Individual axes (such as latitude, longitude, time) can be accessed through the properties of the domain object.

In [54]:
print(f"All axes types available in the domain: {domain.all_axes_types}")

lat_axis = domain.latitude
lon_axis = domain.longitude
time_axis = domain.time

print(f"\nCoordinate name: {lat_axis.name}")
print(f"Values: {lat_axis.values}")

print(f"\nCoordinate name: {lon_axis.name}")
print(f"Values: {lon_axis.values}")

print(f"\nCoordinate name: {time_axis.name}")
print(f"Values: {time_axis.values}")

All axes types available in the domain: [<AxisType.TIME: 'time'>, <AxisType.LATITUDE: 'latitude'>, <AxisType.LONGITUDE: 'longitude'>]

Coordinate name: latitude
Values: [-90. -45.   0.  45.  90.]

Coordinate name: longitude
Values: [-180.  -90.    0.   90.  180.]

Coordinate name: time
Values: ['2000-01-01T00:00:00.000000000']


### Axis Size and Axis Presence Check

In [55]:
from climatrix.dataset.axis import AxisType

print(f"Latitude axis size: {domain.get_size(AxisType.LATITUDE)}")
print(f"Longitude axis size: {domain.get_size('longitude')}")
print(f"Time axis size: {domain.get_size(AxisType.TIME)}")

print(f"\nDoes the domain have a time axis? {domain.has_axis(AxisType.POINT)}")
print(f"Does the domain have a vertical axis? {domain.has_axis(AxisType.VERTICAL)}")

Latitude axis size: 5
Longitude axis size: 5
Time axis size: 1

Does the domain have a time axis? False
Does the domain have a vertical axis? False


### Domain shape and size

In [56]:
print(f"Domain shape (dimensions): {domain.shape}")
print(f"Total domain size (number of data points): {domain.size}")

Domain shape (dimensions): (5, 5)
Total domain size (number of data points): 25


### Arithmetic Operations
BaseClimatrixDataset supports basic arithmetic operations.

In [57]:
print(f"Orginal dataset:\n{cm_dense.da}")

cm_plus_10 = cm_dense + 10
print(f"\nDataset after adding 10:\n{cm_plus_10.da}")

cm_multiplied = cm_dense * 2 #
print(f"\nDataset after multiplying by 2:\n{cm_multiplied.da}")

Orginal dataset:
<xarray.DataArray 'temperature' (latitude: 5, longitude: 5)> Size: 200B
array([[0.73858323, 0.6263338 , 0.16790515, 0.02675058, 0.31968012],
       [0.82799214, 0.2827106 , 0.97982982, 0.91588845, 0.86449415],
       [0.36546245, 0.92056203, 0.27499391, 0.97504155, 0.46360415],
       [0.42710273, 0.17509717, 0.01181027, 0.25986363, 0.63533498],
       [0.17049364, 0.94538297, 0.56248504, 0.96208993, 0.75380703]])
Coordinates:
    time       datetime64[us] 8B 2000-01-01
  * latitude   (latitude) int64 40B -90 -45 0 45 90
  * longitude  (longitude) int64 40B -180 -90 0 90 180

Dataset after adding 10:
<xarray.DataArray 'temperature' (latitude: 5, longitude: 5)> Size: 200B
array([[10.73858323, 10.6263338 , 10.16790515, 10.02675058, 10.31968012],
       [10.82799214, 10.2827106 , 10.97982982, 10.91588845, 10.86449415],
       [10.36546245, 10.92056203, 10.27499391, 10.97504155, 10.46360415],
       [10.42710273, 10.17509717, 10.01181027, 10.25986363, 10.63533498],
       

## Creating a Domain Using `Domain.from_lat_lon`

In [58]:
from climatrix.dataset.domain import Domain
from climatrix.dataset.axis import AxisType

dense_domain_from_func = Domain.from_lat_lon(
    lat=slice(40, 50, 1),
    lon=slice(-100, -90, 1),
    kind='dense'
)
print("Dense domain created using `from_lat_lon`:")
print(f"Type: {type(dense_domain_from_func)}")
print(f"Latitudes: {dense_domain_from_func.latitude.values}")
print(f"Longitudes: {dense_domain_from_func.longitude.values}")
print(f"Is sparse? {dense_domain_from_func.is_sparse}")

sparse_lat = np.array([40.5, 42.1, 45.3])
sparse_lon = np.array([-95.2, -93.0, -90.5])
sparse_domain_from_func = Domain.from_lat_lon(
    lat=sparse_lat,
    lon=sparse_lon,
    kind='sparse'
)
print("\nSparse domain created using `from_lat_lon`:")
print(f"Type: {type(sparse_domain_from_func)}")
print(f"Latitudes: {sparse_domain_from_func.latitude.values}")
print(f"Longitudes: {sparse_domain_from_func.longitude.values}")
print(f"Is sparse? {sparse_domain_from_func.is_sparse}")
print(f"Size of 'point' axis: {sparse_domain_from_func.get_size(AxisType.POINT)}")


Dense domain created using `from_lat_lon`:
Type: <class 'climatrix.dataset.domain.DenseDomain'>
Latitudes: [40. 41. 42. 43. 44. 45. 46. 47. 48. 49. 50.]
Longitudes: [-100.  -99.  -98.  -97.  -96.  -95.  -94.  -93.  -92.  -91.  -90.]
Is sparse? False

Sparse domain created using `from_lat_lon`:
Type: <class 'climatrix.dataset.domain.SparseDomain'>
Latitudes: [40.5 42.1 45.3]
Longitudes: [-95.2 -93.  -90.5]
Is sparse? True
Size of 'point' axis: 3
